In [1]:
import pandas as pd
import numpy as np
from supervised.automl import AutoML
from dateutil.relativedelta import relativedelta
from sklearn.metrics import roc_auc_score

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## DF Original

In [2]:
df_canada = pd.read_csv('canada_updated.csv')
df_canada["date"] = pd.to_datetime(df_canada["date"])
df_canada.head()

,date,QUALITY_FLAG,cid,industry_raw,E_TTM_period_date,E_TTM_ammor_intangibles,E_TTM_asset_writedown,E_TTM_assets_gro_five,E_TTM_capex,E_TTM_cash_acquisitions,...,E_G_ebitda_cov,E_G_ret_on_asset,E_G_ret_on_inv_cap,E_G_net_to_cash,E_G_perm_assets_ratio,return_1q,target_net_income,target_cash_operations,binary_target_net_income,binary_target_cash_operations
0,2002-01-03,True,SP-065996,NaN,2001-10-31,0.000,0.0,0.0,-12.738000,-3.336,...,-165.453488,0.130018,0.101871,-0.068216,0.414230,NaN,NaN,NaN,0,0
1,2002-01-08,True,SP-002396,NaN,2001-09-30,3.078,0.0,0.0,-20.889000,-68.220,...,-2.685925,0.071119,0.067430,-0.004881,0.595752,NaN,NaN,NaN,0,0
2,2002-01-08,True,SP-006704,NaN,2001-09-30,0.000,0.0,0.0,-17.971623,0.000,...,2.262460,-0.069781,-0.039238,-0.045993,0.775432,NaN,NaN,NaN,0,0
3,2002-01-08,True,SP-008644,NaN,2001-09-30,0.000,0.0,0.0,-34.700000,0.000,...,-4.852273,-0.169833,-0.155712,-0.316372,0.773996,NaN,NaN,NaN,0,0
4,2002-01-08,True,SP-013994,NaN,2001-09-30,0.000,0.0,0.0,-1403.000000,-133.000,...,-14.569697,0.109798,0.078497,-0.157934,0.921832,NaN,NaN,NaN,0,0


In [3]:
df_canada["industry_raw"].unique()

array([nan, 'Ground Transportation', 'Banks',
       'Semiconductor Equipment & Products', 'Specialty Retail',
       'Financial Services', 'Communications Equipment',
       'Electronic Equipment, Instruments & Components',
       'Passenger Airlines', 'Machinery',
       'Commercial Services & Supplies', 'Automobile Components', 'Media',
       'Consumer Staples Distribution & Retail', 'Metals & Mining',
       'Software', 'Health Care Providers & Services', 'Biotechnology',
       'Hotels, Restaurants & Leisure', 'Electric Utilities',
       'Pharmaceuticals', 'Internet Software & Services', 'Chemicals',
       'IT Services', 'Insurance', 'Containers & Packaging',
       'Oil, Gas & Consumable Fuels', 'Industrial Conglomerates',
       'Trading Companies & Distributors', 'Paper & Forest Products',
       'Energy Equipment & Services',
       'Diversified Telecommunication Services', 'Broadline Retail',
       'Aerospace & Defense', 'Construction & Engineering',
       'Technology Ha

In [4]:
df_canada["industry_raw"].value_counts()

industry_raw
Metals & Mining                8554
Oil, Gas & Consumable Fuels    7239
Energy Equipment & Services    1770
Media                          1424
Capital Markets                1266
                               ... 
Office REITs                     16
Industrial REITs                 14
Household Products               13
Specialized REITs                 7
Health Care REITs                 7
Name: count, Length: 76, dtype: int64

## Copie pour modification

In [5]:
df_model = df_canada.copy()

df_model['date'] = pd.to_datetime(df_model['date'], errors='coerce')
df_model.sort_values(by=['cid', 'date'], inplace=True)

# Retirer les lignes où Quality_Flag est False
df_model = df_model[df_model['QUALITY_FLAG'] == True]

# (FACULTATIF) Exclure les banques
# df_model = df_model[df_model['industry'] != 'Banks']

## Selection de features

In [6]:
def select_features(
    df,
    include_agro=False,
    include_rgro=False,
    include_tcgro=False,
    include_ratios_assets=False,
    include_ratios_rev=False,
    include_ratios_totcap=False,
    mandatory_cols=None
):
    """
    Sélectionne dynamiquement les colonnes d'un DataFrame en fonction
    des familles de variables explicatives demandées,
    en plaçant d'abord les colonnes obligatoires (mandatory_cols).
    """
    
    # 1) Gérer la liste mandatory_cols (par défaut : vide ou ['cid','date'] selon besoin)
    if mandatory_cols is None:
        mandatory_cols = []
    
    # 2) Définir les "familles" de motifs
    family_patterns = {
        'agro': ['_agro_1q', '_agro_4q'],
        'rgro': ['_rgro_1q', '_rgro_4q'],
        'tcgro': ['_tcgro_1q', '_tcgro_4q'],
        'ratios_assets': ['_on_assets_ratio'],
        'ratios_rev': ['_on_rev_ratio'],
        'ratios_totcap': ['_on_tot_cap_ratio']
    }
    
    # 3) Construire la liste des motifs à inclure
    patterns_to_keep = []
    if include_agro:
        patterns_to_keep += family_patterns['agro']
    if include_rgro:
        patterns_to_keep += family_patterns['rgro']
    if include_tcgro:
        patterns_to_keep += family_patterns['tcgro']
    
    if include_ratios_assets:
        patterns_to_keep += family_patterns['ratios_assets']
    if include_ratios_rev:
        patterns_to_keep += family_patterns['ratios_rev']
    if include_ratios_totcap:
        patterns_to_keep += family_patterns['ratios_totcap']
    
    # 4) Retrouver toutes les colonnes du df qui matchent nos motifs
    matched_cols = set()
    for pat in patterns_to_keep:
        for col in df.columns:
            if pat in col:
                matched_cols.add(col)
    # => matched_cols est un set() de colonnes
    
    # 5) Conserver l'ordre original des colonnes matched, 
    #    en filtrant df.columns dans l'ordre d'origine
    matched_cols_in_order = [c for c in df.columns if c in matched_cols]
    
    # 6) Construire l'ordre final :
    #    - d'abord mandatory_cols (dans l'ordre donné),
    #    - puis les matched_cols (dans l'ordre d'origine)
    #    - attention aux colonnes obligatoires qui n'existent pas, 
    #      ou aux duplications
    #    - on fait donc une intersection + un set() pour éviter 
    #      les collisions.
    
    # Intersection pour ne pas inclure des mandatory inexistantes
    mandatory_cols_in_df = [c for c in mandatory_cols if c in df.columns]
    
    # Puis on concatène en évitant toute duplication
    columns_to_keep_ordered = mandatory_cols_in_df + [
        c for c in matched_cols_in_order if c not in mandatory_cols_in_df
    ]
    
    # 7) Extraire le sous-DataFrame
    df_filtered = df[columns_to_keep_ordered].copy()
    
    # 8) (Optionnel) trier par cid/date si elles sont présentes
    if 'cid' in df_filtered.columns and 'date' in df_filtered.columns:
        df_filtered.sort_values(by=['cid', 'date'], inplace=True)
    
    return df_filtered

In [7]:
df_model_final = select_features(
    df_model,
    include_agro=True,
    include_rgro=False,
    include_tcgro=False,
    include_ratios_assets=True,
    include_ratios_rev=False,
    include_ratios_totcap=False,
    mandatory_cols=['cid', 'date', 'binary_target_net_income']  # je garde la target
)

# Compter le nombre total de NaN dans tout le DataFrame
total_nan = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans df_model_final : {total_nan}")

# Retirer les lignes qui contiennent AU MOINS un NaN
df_model_final.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

print(df_model_final.shape)

Nombre total de valeurs NaN dans df_model_final : 443777
Nombre total de valeurs NaN après suppression : 0
(37311, 195)


## DF Test pour tester la fonction

In [8]:
# Date la plus récente du DataFrame
max_date = df_model_final['date'].max()

# Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=5)

# Filtrer pour ne garder que les 5 dernières années
df_test = df_model_final[df_model_final['date'] >= cutoff_date].copy()

print(df_test['date'].min(), df_test['date'].max())
print(df_test.shape)

2019-12-12 00:00:00 2024-12-12 00:00:00
(7552, 195)


## Rolling Window + AutoML

In [9]:
def pipeline_rolling_windows(data, date_col, target_col, train_years, val_years, test_years, buffer_months=0):
    """
    Pipeline direct pour la rolling window avec AutoML et cross-validation personnalisée.
    Ajoute les périodes dans le DataFrame final pour validation.
    """
    # Conversion de la colonne date
    data[date_col] = pd.to_datetime(data[date_col])
    start_date = data[date_col].min()
    end_date = data[date_col].max()

    predictions_all = []  # Liste pour stocker toutes les prédictions

    while start_date + relativedelta(years=train_years + val_years + test_years) <= end_date:
        # Définir les périodes
        train_end = start_date + relativedelta(years=train_years) - pd.Timedelta(days=1)
        tampon_1_end = train_end + relativedelta(months=buffer_months)
        val_start = tampon_1_end + pd.Timedelta(days=1)
        val_end = val_start + relativedelta(years=val_years) - pd.Timedelta(days=1)
        tampon_2_end = val_end + relativedelta(months=buffer_months)
        test_start = tampon_2_end + pd.Timedelta(days=1)
        test_end = test_start + relativedelta(years=test_years) - pd.Timedelta(days=1)

        # Filtrer les données
        train_data = data.loc[(data[date_col] >= start_date) & (data[date_col] <= train_end)]
        val_data = data.loc[(data[date_col] >= val_start) & (data[date_col] <= val_end)]
        test_data = data.loc[(data[date_col] >= test_start) & (data[date_col] <= test_end)]

        if len(train_data) == 0 or len(val_data) == 0 or len(test_data) == 0:
            print(f"Fenêtre {start_date.year}-{test_end.year} : données insuffisantes, sautée.")
            start_date += relativedelta(years=1)
            continue

        # Configurer et entraîner AutoML
        print(f"Fenêtre {start_date.year}-{test_end.year} : entraînement de AutoML...")
        automl = AutoML(results_path=f"AutoML_{start_date.year}-{test_end.year}", mode="Perform", algorithms=["Xgboost"], eval_metric="auc")
        custom_cv = [(train_data.index, val_data.index)]
        automl.fit(
            train_data.drop(columns=[target_col, date_col, 'cid']),
            train_data[target_col], cv=custom_cv
        )

        # Prédire sur le test set
        test_preds = test_data[[date_col, target_col]].copy()
        #test_preds["predicted"] = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        proba = automl.predict_proba(test_data.drop(columns=[target_col, date_col, 'cid']))
        test_preds["prob_down"] = proba[:, 0]  # Probabilité pour la classe 0 (diminution des bénéfices)
        test_preds["prob_up"] = proba[:, 1]  # Probabilité pour la classe 1 (augmentation des bénéfices)
        test_preds["net_prob_score"] = test_preds["prob_up"] - test_preds["prob_down"]
        test_preds["window"] = f"{start_date.year}-{test_end.year}"
        test_preds["cid"] = test_data["cid"].values

        # Ajouter les périodes pour validation
        #test_preds["train_start"] = start_date
        #test_preds["train_end"] = train_end
        #test_preds["tampon_1"] = tampon_1_end
        #test_preds["val_start"] = val_start
        #test_preds["val_end"] = val_end
        #test_preds["tampon_2"] = tampon_2_end
        #test_preds["test_start"] = test_start
        #test_preds["test_end"] = test_end

        # Sauvegarder les prédictions
        predictions_all.append(test_preds)

        # Avancer la fenêtre
        start_date += relativedelta(years=1)

    predictions_df = pd.concat(predictions_all, ignore_index=True)
    return predictions_df

## Appel de fonction

In [10]:
predictions_df = pipeline_rolling_windows(
    data=df_test, 
    date_col="date", 
    target_col="binary_target_net_income", 
    train_years=2, 
    val_years=1, 
    test_years=1, 
    buffer_months=1
)

Fenêtre 2019-2024 : entraînement de AutoML...
AutoML directory: AutoML_2019-2024
The task is binary_classification with evaluation metric auc
AutoML will use algorithms: ['Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model
1_Default_Xgboost auc 0.866391 trained in 18.24 seconds (1-sample predict time 0.0343 seconds)
* Step not_so_random will try to check up to 4 models
2_Xgboost auc 0.860464 trained in 13.96 seconds (1-sample predict time 0.0348 seconds)
3_Xgboost auc 0.86656 trained in 14.81 seconds (1-sample predict time 0.0341 seconds)
4_Xgboost auc 0.844248 trained in 15.64 seconds (1-sample predict time 0.0353 seconds)
5_Xgboost auc 0.795209 trained in 8.74 seconds (1-sample pred

In [11]:
predictions_df.to_csv("df_test_2025.csv", index=False)

## Grouped AUCs

In [12]:
def compute_auc_by_groups(
    predictions_df: pd.DataFrame, 
    df_canada: pd.DataFrame,
    group_cols: list,
    cid_col: str = "cid",
    date_col: str = "date",
    target_col: str = "binary_target_net_income",
    prob_col: str = "prob_up",
    out_csv: str = None
):
    """
    Calcule l'AUC par groupes (selon group_cols) et le nombre de lignes dans chaque groupe.
    
    Arguments :
    -----------
    - predictions_df : DataFrame contenant au moins [cid_col, date_col, target_col, prob_col]
    - df_canada : DataFrame contenant au moins [cid_col, date_col, 'return_1q', 'industry_raw', etc.]
    - group_cols : liste des colonnes sur lesquelles faire le groupby (ex: ["year", "industry_raw"])
    - cid_col : nom de la colonne identifiant l'entreprise (défaut: "cid")
    - date_col : nom de la colonne date (défaut: "date")
    - target_col : nom de la colonne cible binaire (ex: 0/1) (défaut: "binary_target_net_income")
    - prob_col : nom de la colonne contenant la probabilité pour la classe positive (défaut: "prob_up")
    - out_csv : chemin vers le fichier CSV de sortie (défaut: None, ne pas exporter)

    Retour :
    --------
    - final_df : DataFrame avec AUC et Count pour chaque groupe
    """

    # 1) Fusionner les dataframes (left merge pour conserver toutes les prédictions)
    df_merged = predictions_df.merge(
        df_canada[[cid_col, date_col, 'return_1q', 'industry_raw']],  # adapt si tu veux plus/moins de colonnes
        on=[cid_col, date_col],
        how='left'
    )

    # 2) Si "year" est dans group_cols et pas encore créée, on l'extrait du date_col
    if "year" in group_cols and "year" not in df_merged.columns:
        df_merged["year"] = pd.to_datetime(df_merged[date_col]).dt.year

    # -- Fonction interne pour calculer l’AUC sur un sous-groupe
    def group_auc(sub_df):
        unique_targets = sub_df[target_col].unique()
        # Gérer le cas où il n’y a qu’une seule classe (roc_auc_score plantait sinon)
        if len(unique_targets) < 2:
            return float("nan")
        return roc_auc_score(sub_df[target_col], sub_df[prob_col])

    # 3) Calculer l’AUC par groupe
    grouped_aucs = (
        df_merged
        .groupby(group_cols)
        .apply(group_auc)
        .reset_index(name="AUC")
    )

    # 4) Calculer le nombre d’observations par groupe (ou nombre de cid uniques, selon besoin)
    #    Ici : on compte simplement le nombre de lignes
    grouped_counts = (
        df_merged
        .groupby(group_cols)[target_col]  # ou [cid_col] si tu veux le nombre d'entreprises distinctes
        .count()
        .reset_index(name="Count")
    )

    # 5) Fusionner AUC et Count
    final_df = pd.merge(grouped_aucs, grouped_counts, on=group_cols, how="left")

    # Optionnel : trier le résultat
    final_df = final_df.sort_values(group_cols)

    # Optionnel : exporter en CSV
    if out_csv is not None:
        final_df.to_csv(out_csv, index=False)

    return final_df

In [13]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["year", "industry_raw"],
    out_csv="auc_par_annee_et_industrie.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,year,industry_raw,AUC,Count
0,2023,Aerospace & Defense,0.987500,18
1,2023,Air Freight & Logistics,NaN,4
2,2023,Automobile Components,0.600000,19
3,2023,Banks,0.655797,35
4,2023,Beverages,1.000000,3
...,...,...,...,...
113,2024,Specialty Retail,NaN,26
114,2024,"Textiles, Apparel & Luxury Goods",NaN,8
115,2024,Trading Companies & Distributors,NaN,31
116,2024,Transportation Infrastructure,NaN,4


In [14]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["industry_raw"],
    out_csv="auc_par_industrie.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,industry_raw,AUC,Count
0,Aerospace & Defense,0.901786,36
1,Air Freight & Logistics,NaN,8
2,Automobile Components,0.590909,37
3,Banks,0.816038,65
4,Beverages,0.700000,7
5,Biotechnology,0.166667,10
6,Broadline Retail,0.236364,16
7,Capital Markets,0.676721,86
8,Chemicals,0.687500,55
9,Commercial Services & Supplies,0.631452,82


In [15]:
compute_auc_by_groups(
    predictions_df=predictions_df,
    df_canada=df_canada,
    group_cols=["year"],
    out_csv="auc_par_annee.csv"
)

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/2025299659.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,year,AUC,Count
0,2023,0.716096,1428
1,2024,NaN,1416


## Création de Portefeuille + Annual Weighted Returns

In [16]:
def create_weighted_portfolios(predictions_df, df_canada, proba_col, return_col, lower_threshold=0.4, upper_threshold=0.6):
    """
    Crée un portefeuille pondéré basé sur les prédictions et calcule les rendements pondérés par année,
    en attribuant des poids positifs aux positions longues et négatifs aux positions courtes, avec une somme neutre.
    
    Args:
    - predictions_df (pd.DataFrame): DataFrame contenant les prédictions et les identifiants 'cid'.
    - df_canada (pd.DataFrame): DataFrame contenant les rendements futurs et les identifiants 'cid'.
    - return_col (str): Nom de la colonne des rendements futurs dans df_canada.
    - lower_threshold (float): Seuil inférieur pour les positions courtes.
    - upper_threshold (float): Seuil supérieur pour les positions longues.
    
    Returns:
    - result_df (pd.DataFrame): DataFrame contenant les rendements pondérés des portefeuilles par année.
    """
    # Assure que les colonnes 'date' sont au bon format datetime dans les deux DataFrames
    predictions_df['date'] = pd.to_datetime(predictions_df['date'])
    df_canada['date'] = pd.to_datetime(df_canada['date'])
        
    
    # Joindre les deux DataFrames sur 'cid' et 'date'
    merged_df = predictions_df.merge(df_canada[['cid', 'date', return_col]], on=['cid', 'date'], how='left')
    
    # Extraire l'année à partir de la colonne 'date'
    merged_df['year'] = merged_df['date'].dt.year
    
    # Retirer les lignes avec des valeurs manquantes
    merged_df.dropna(inplace=True)
    
    # Initialiser une liste pour stocker les résultats
    results = []

    # Grouper par année
    for year, group in merged_df.groupby('year'):
        # Sélectionner les positions longues et courtes selon les seuils
        selected_long = group[group[proba_col] > upper_threshold]  # Positions longues
        selected_short = group[group[proba_col] < lower_threshold]  # Positions courtes
        
        n_long = len(selected_long)
        n_short = len(selected_short)
        
        if n_long > 0 or n_short > 0:
            # Attribuer des poids aux positions longues et courtes
            if n_long > 0:
                selected_long.loc[:, 'weight'] = 1 / n_long  # Poids positifs pour les positions longues
            if n_short > 0:
                selected_short.loc[:, 'weight'] = -1 / n_short  # Poids négatifs pour les positions courtes
            
            # Combiner les deux DataFrames
            selected = pd.concat([selected_long, selected_short], ignore_index=True)
            
            # Calculer le rendement pondéré du portefeuille (somme des rendements pondérés)
            weighted_return = (selected['weight'] * selected[return_col] * 100).sum()
            
            # Ajouter le résultat à la liste
            results.append({'year': year, 'weighted_return': weighted_return})
        else:
            # Si aucune action ne respecte les seuils, le rendement est NaN
            results.append({'year': year, 'weighted_return': float('nan')})

    # Convertir les résultats en DataFrame
    result_df = pd.DataFrame(results)
    
    return result_df

## Appel de fonction rendement annuel

In [17]:
ret_net_income = create_weighted_portfolios(predictions_df, df_canada, 'net_prob_score', 'return_1q', lower_threshold=0.5, upper_threshold=0.5)

#ret_net_income.to_csv("returns_net_income_0.5_0.5.csv", index=False)

ret_net_income

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2023,-0.088229
1,2024,-1.622741


In [18]:
returns_net_income = create_weighted_portfolios(predictions_df, df_canada, 'net_prob_score', 'return_1q', lower_threshold=0.4, upper_threshold=0.6)

#returns_net_income.to_csv("returns_net_income_0.4_0.6.csv", index=False)

returns_net_income

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_1219/444120012.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,year,weighted_return
0,2023,-0.725660
1,2024,-1.766371


## Rendement par décile

In [19]:
def compute_decile_returns(
    predictions_df: pd.DataFrame,
    df_canada: pd.DataFrame,
    proba_col: str,
    return_col: str,
    aggregator: str = "mean",        # "mean" ou "median"
    decile_weights: list = None,     # liste de 10 poids ou None
    csv_path: str = None             # chemin CSV ou None
) -> pd.DataFrame:
    """
    Calcule le rendement par décile (du plus faible au plus élevé) pour chaque année.
    - Découpage en déciles via pd.qcut (q=10).
    - Agrégation des rendements par décile via un 'aggregator' (mean ou median).
    - Optionnellement, applique un vecteur de poids (decile_weights) pour calculer
      un rendement global (somme pondérée des déciles).

    Paramètres
    ----------
    predictions_df : pd.DataFrame
        Contient les prédictions (probabilités) et les colonnes 'cid', 'date'.
        Doit inclure la colonne `proba_col`.
    df_canada : pd.DataFrame
        Contient la colonne `return_col` ainsi que 'cid', 'date'.
    proba_col : str
        Nom de la colonne dans predictions_df représentant la probabilité ou le score.
    return_col : str
        Nom de la colonne dans df_canada qui contient le rendement (ou variation).
    aggregator : {"mean", "median"}, défaut = "mean"
        Choix entre la moyenne ou la médiane comme statistique de rendement pour le décile.
    decile_weights : list of float (length 10) ou None
        Si fourni, liste de 10 poids (un pour chaque décile de 0 à 9).
        Permet de calculer une colonne "portfolio_return" = somme(décile_i * poids_i).
    csv_path : str ou None
        Si un chemin est fourni, le DataFrame final sera sauvegardé en CSV.

    Retour
    ------
    df_final : pd.DataFrame
        DataFrame pivoté avec:
        - 1 ligne par année
        - 10 colonnes (D1 à D10) indiquant le rendement agrégé de chaque décile
        - Si decile_weights est fourni, une colonne supplémentaire 'portfolio_return'
          représentant la somme pondérée des rendements déciles.
    """

    # --- 1) Merge sur cid + date
    merged_df = predictions_df.merge(
        df_canada[['cid', 'date', return_col]],
        on=['cid', 'date'],
        how='left'
    )
    
    # --- 2) Extraire l'année
    merged_df['year'] = merged_df['date'].dt.year
    
    # --- 3) Nettoyage éventuel
    merged_df.dropna(subset=[proba_col, return_col, 'year'], inplace=True)
    
    # --- 4) Choix de la fonction d'agrégation
    if aggregator == "mean":
        aggregator_func = np.mean
    elif aggregator == "median":
        aggregator_func = np.median
    else:
        raise ValueError("aggregator must be 'mean' or 'median'.")
    
    # --- 5) Préparer une liste pour stocker (year, decile_bin, agg_return)
    decile_results = []
    
    for year, group in merged_df.groupby('year'):
        # On doit découper en 10 déciles
        group = group.copy()
        
        # Astuce: labels=False renvoie des labels 0..9
        group['decile_bin'] = pd.qcut(group[proba_col], q=10, labels=False)
        
        # Calculer l'agrégat (mean ou median) du rendement dans chaque décile
        for decile_id, subdf in group.groupby('decile_bin'):
            if pd.isnull(decile_id):
                continue
            
            decile_id = int(decile_id)
            # Rendement agrégé (en % si on veut)
            decile_value = aggregator_func(subdf[return_col]) * 100.0
            
            decile_results.append({
                'year': year,
                'decile': decile_id,
                'decile_return': decile_value
            })
    
    # --- 6) On transforme la liste en DataFrame + pivot
    decile_df = pd.DataFrame(decile_results)
    df_pivot = decile_df.pivot(index='year', columns='decile', values='decile_return')
    
    # Renommer les colonnes pour D1..D10
    # Attention : decile_id = 0 correspond au 1er décile => D1
    df_pivot.columns = [f"D{i+1}" for i in df_pivot.columns]
    
    # --- 7) Si on a des poids, on calcule la somme pondérée
    if decile_weights is not None:
        if len(decile_weights) != 10:
            raise ValueError("decile_weights must be a list of length 10.")
        
        # Multiplier chaque colonne D1..D10 par le poids correspondant, puis sommer
        # Astuce : df_pivot.iloc[:, 0:10] prend les 10 premières colonnes => D1..D10
        # On peut ensuite .mul(decile_weights, axis='columns') pour multiplier par le vecteur
        # et .sum(axis=1) pour obtenir la somme par année
        # (on suppose que l'ordre des colonnes est D1..D10).
        
        weighted_sum = df_pivot.iloc[:, 0:10].mul(decile_weights, axis='columns').sum(axis=1)
        
        # On l'ajoute comme nouvelle colonne
        df_pivot['portfolio_return'] = weighted_sum
    
    # --- 8) On remet l'index 'year' comme colonne
    df_final = df_pivot.reset_index()
    
    # --- 9) Sauvegarde CSV si demandé
    if csv_path is not None:
        df_final.to_csv(csv_path, index=False)
    
    return df_final

In [28]:
df_deciles = compute_decile_returns(
    predictions_df=predictions_df,
    df_canada=df_canada,
    proba_col='net_prob_score',
    return_col='return_1q',
    aggregator='mean',
    decile_weights=[-0.1, -0.1, 0, 0, 0.1, 0.2, 0.1, 0.2, 0.2, 0.2],
    csv_path='my_decile_returns.csv'
)

df_deciles.head()

,year,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,portfolio_return
0,2023,0.522507,0.683460,1.737135,2.468283,5.336049,3.263129,2.187534,4.934321,0.774357,0.919068,2.609937
1,2024,4.591108,6.823602,4.750542,3.774460,4.064164,4.586458,2.925520,4.973091,3.571028,0.919241,2.367461


## Preuve CSV Rendement par Décile

In [27]:
# --- 1) Merge sur cid + date
merged_df = predictions_df.merge(
    df_canada[['cid', 'date', 'return_1q']],
    on=['cid', 'date'],
    how='left'
    )

# --- 2) Extraire l'année
merged_df['year'] = merged_df['date'].dt.year

# Filtrer pour l'année 2023
year = 2023
year_df = merged_df[merged_df['year'] == year].copy()

# Attribuer les déciles pour cette année
year_df['decile_bin'] = pd.qcut(year_df['net_prob_score'], q=10, labels=False)

# Liste pour stocker chaque subdf avec une colonne indiquant le décile
list_of_subdfs = []

# Itérer sur chaque décile et collecter les sous-ensembles
for decile_id, subdf in year_df.groupby('decile_bin'):
    # Ajouter une colonne indiquant le décile courant
    subdf = subdf.copy()
    subdf['decile_id'] = decile_id
    list_of_subdfs.append(subdf)

# Concaténer tous les subdf en un seul DataFrame
all_subdfs_df = pd.concat(list_of_subdfs, ignore_index=True)

# Exporter vers CSV
all_subdfs_df.to_csv("subdfs_2023.csv", index=False)

# BackUp

In [21]:
def create_weighted_portfolios_backup(
    predictions_df, 
    df_canada, 
    proba_col, 
    return_col, 
    lower_threshold=0.4, 
    upper_threshold=0.6,
    group_mode=None,  # "decile" ou "binning" ou None
    return_decile_or_bins=False,
    debug_bins=False  # <-- Paramètre pour afficher les stats des bins 
):
    """
    Exemple d'une fonction qui calcule un portefeuille
    + rendements par déciles ou binning.
    Et qui, si debug_bins=True, affiche la répartition des probabilités dans chaque bin.
    """
    
    merged_df = predictions_df.merge(df_canada[['cid', 'date', return_col]], 
                                     on=['cid', 'date'], how='left')
    
    merged_df['year'] = merged_df['date'].dt.year
    merged_df.dropna(subset=[proba_col, return_col, 'year'], inplace=True)
    
    # --- 1) Calcul du portefeuille "classique" (logique seuils) ---
    results = []
    for year, group in merged_df.groupby('year'):
        selected_long = group[group[proba_col] > upper_threshold].copy()
        selected_short = group[group[proba_col] < lower_threshold].copy()
        
        n_long = len(selected_long)
        n_short = len(selected_short)
        
        if n_long > 0 or n_short > 0:
            if n_long > 0:
                selected_long['weight'] = 1.0 / n_long
            if n_short > 0:
                selected_short['weight'] = -1.0 / n_short
            
            selected = pd.concat([selected_long, selected_short], ignore_index=True)
            weighted_return = (selected['weight'] * selected[return_col] * 100).sum()
            
            results.append({'year': year, 'weighted_return': weighted_return})
        else:
            results.append({'year': year, 'weighted_return': float('nan')})
    
    result_df = pd.DataFrame(results)
    
    # --- 2) Calcul par déciles ou binning (optionnel) ---
    decile_or_bin_returns_df = None
    if return_decile_or_bins and group_mode is not None:
        decile_or_bin_list = []
        
        for year, group in merged_df.groupby('year'):
            group = group.copy().reset_index(drop=True)
            
            # Construire les bins/déciles
            if group_mode == "decile":
                group['decile_bin'] = pd.qcut(group[proba_col], q=10, labels=False)
            elif group_mode == "binning":
                bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
                group['decile_bin'] = pd.cut(group[proba_col], bins=bins, labels=False, include_lowest=True)
            else:
                raise ValueError("group_mode must be either 'decile' or 'binning'")
            
            # -- Si on veut déboguer / visualiser les bins --
            #if debug_bins:
                #print(f"\n=== Année {year} : Répartition des probabilités par bin ===")
                # Regrouper par decile_bin et afficher min, max, count
                #bin_stats = group.groupby('decile_bin')[proba_col].agg(['min','max','count'])
                #print(bin_stats)
                #bin_all_probas = group.groupby('decile_bin')[proba_col].apply(list)
                #print(bin_all_probas)
            
            # Calculer le rendement par bin/décile
            for bin_id, subdf in group.groupby('decile_bin'):
                if pd.isnull(bin_id) or len(subdf) == 0:
                    continue
                
                bin_id = int(bin_id)
                
                # Déterminer si ce bin/décile est long ou short via la moyenne
                proba_mean = subdf[proba_col].mean()
                if proba_mean < 0.5:
                    w = -1.0 / len(subdf)
                else:
                    w = 1.0 / len(subdf)
                
                bin_return = (subdf[return_col] * w * 100).sum()
                
                decile_or_bin_list.append({
                    'year': year,
                    'decile_bin': bin_id,
                    'weighted_return': bin_return
                })
        
        decile_or_bin_df = pd.DataFrame(decile_or_bin_list)
        decile_or_bin_returns_df = decile_or_bin_df.pivot(index='year', 
                                                          columns='decile_bin', 
                                                          values='weighted_return')
        
        # Renommer les colonnes
        if group_mode == "decile":
            decile_or_bin_returns_df.columns = [f'D{c+1}' for c in decile_or_bin_returns_df.columns]
        else:
            decile_or_bin_returns_df.columns = [f'Bin{c}' for c in decile_or_bin_returns_df.columns]
        
        decile_or_bin_returns_df.reset_index(inplace=True)
    
        return result_df, decile_or_bin_returns_df
    else:
        return result_df

In [23]:
test_binning = create_weighted_portfolios_backup(predictions_df, df_canada, 'net_prob_score', 'return_1q', lower_threshold=-0.5, upper_threshold=0.6, group_mode="decile", return_decile_or_bins=True, debug_bins=True)

#test_binning.to_csv("returns_net_income_0.5_0.5.csv", index=False)

test_binning

(   year  weighted_return
 0  2023         0.492533
 1  2024        -3.000721,
    year        D1        D2        D3        D4        D5        D6        D7  \
 0  2023 -0.522507 -0.683460 -1.737135 -2.468283 -5.336049 -3.263129 -2.187534   
 1  2024 -4.591108 -6.823602 -4.750542 -3.774460 -4.064164 -4.586458 -2.925520   
 
          D8        D9       D10  
 0  4.934321  0.774357  0.919068  
 1  4.973091  3.571028  0.919241  )